**ATTENTION: TO BE ABLE TO RUN ANY OF THE BELOW CODE --->>>> THE 'eece490proj' FOLDER SHOULD BE PLACED DIRECTLY IN THE C: FOLDER OF THE PC**

**PLEASE DOWNLOAD THE LIBRARIES FOUND IN THE requirements.txt FILE**

**THIS IS THE TRAINING CODE .... THE INFERENCE CODE IS 'inference.ipynb'**

**FIRST CODE: IT'S JUST FOR CONVERTING ANNOTATIONS FROM ONE SINGLE .GEOJSON FILE -> TO .TXT FILES FOR EACH IMAGE IN THE DATASET**

NO NEED TO RUN IT BECAUSE WE SUCCESSFULLY DID THE CONVERSION

CONTEXT: THE DATASET HAD 'xView_train.geojson' as one label file so we converted it to YOLO annotations format

In [ ]:
#-----------------------------------------------------WE ALREADY DID THE CONVERSION AND PROVIDED THE CONVERTED VERSION--------------------------------------------------
#
#
#RESULT AFTER RUNNING:IN THE xView Folder YOU SHOULD SEE AN IMAGES FOLDER WITH TRAINING AND 
#VALIDATION FOLDERS COTAINING .tif IMAGES, AND 2 autosplit_train & val TXT FILES AND YOU SHOULD 
#SEE IN THE xView FOLDER A FOLDER CALLED LABELS THAT HAS A TRAIN FOLDER THAT CONTAINS TXT FILES

'''

import json
import os
from pathlib import Path

import numpy as np
from PIL import Image
from tqdm import tqdm
from ultralytics.data.utils import autosplit
from ultralytics.utils.ops import xyxy2xywhn


def convert_labels(fname=Path('xView')):
  # Convert xView geoJSON labels to YOLO format
  path = fname.parent
  with open(fname) as f:
      print(f'Loading {fname}...')
      data = json.load(f)
  print(path)
  # Make dirs
  labels = Path(path / 'labels' / 'train')
  os.system(f'rm -rf {labels}')
  labels.mkdir(parents=True, exist_ok=True)

  # xView classes 11-94 to 0-59
  xview_class2index = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 2, -1, 3, -1, 4, 5, 6, 7, 8, -1, 9, 10, 11,
                       12, 13, 14, 15, -1, -1, 16, 17, 18, 19, 20, 21, 22, -1, 23, 24, 25, -1, 26, 27, -1, 28, -1,
                       29, 30, 31, 32, 33, 34, 35, 36, 37, -1, 38, 39, 40, 41, 42, 43, 44, 45, -1, -1, -1, -1, 46,
                       47, 48, 49, -1, 50, 51, -1, 52, -1, -1, -1, 53, 54, -1, 55, -1, -1, 56, -1, 57, -1, 58, 59]

  shapes = {}
  for feature in tqdm(data['features'], desc=f'Converting {fname}'):
      p = feature['properties']
      if p['bounds_imcoords']:
          id = p['image_id']
          file = Path(path / 'images' / 'train' / id)
          if file.exists():  # 1395.tif missing
              try:
                  box = np.array([int(num) for num in p['bounds_imcoords'].split(",")])
                  assert box.shape[0] == 4, f'incorrect box shape {box.shape[0]}'
                  cls = p['type_id']
                  cls = xview_class2index[int(cls)]  # xView class to 0-60
                  assert 59 >= cls >= 0, f'incorrect class index {cls}'

                  # Write YOLO label
                  if id not in shapes:
                      shapes[id] = Image.open(file).size
                  box = xyxy2xywhn(box[None].astype(float), w=shapes[id][0], h=shapes[id][1], clip=True)
                  with open((labels / id).with_suffix('.txt'), 'a') as f:
                      f.write(f"{cls} {' '.join(f'{x:.6f}' for x in box[0])}\n")  # write label.txt

              except Exception as e:
                  print(f'WARNING: skipping one label for {file}: {e}')

dir = Path('xView')  # dataset root dir
# Download manually from https://challenge.xviewdataset.org(only uncomment the below if you do not have the data)
# urls = ['https://d307kc0mrhucc3.cloudfront.net/train_labels.zip',  # train labels
#         'https://d307kc0mrhucc3.cloudfront.net/train_images.zip',  # 15G, 847 train images
#         'https://d307kc0mrhucc3.cloudfront.net/val_images.zip']  # 5G, 282 val images (no labels)
# download(urls, dir=dir)

# Convert labels
convert_labels(dir / 'xView_train.geojson')

# Move images
images = Path(dir / 'images')
images.mkdir(parents=True, exist_ok=True)
Path(dir / 'train_images').rename(dir / 'images' / 'train')
Path(dir / 'val_images').rename(dir / 'images' / 'val')

# Split
autosplit(dir / 'images' / 'train')

'''

**HERE WE CHECK IF CUDA IS PROPERLY CONFIGURED. WE SHOULD FIRST SEE THE VERSION OF CUDA**

In [1]:
#HERE WE CHECK IF CUDA IS PROPERLY CONFIGURED. WE SHOULD FIRST SEE THE VERSION OF CUDA. 
#!pip install ultralytics #IF ANY ERROR UNCOMMENT AND TRY THIS
from ultralytics import YOLO
import torch
import torchvision
import torchaudio

!nvcc --version
print("\n")
#THEN CHECK IF TORCH SUPPORTS CUDA. U SHOULD SEE cudaVersion+cu121 for Cuda 12.1
print(torch.__version__)
print("\n")
#THEN CHECK IF TORCH CAN DETECT GPUS. IF CPU IS PRINTED THEN CUDA CANNOT DETECT GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


2.2.0+cu121


cuda


**HERE IS THE TRAINING CODE**

In [3]:
##### Import the downloaded model from xView Yolo Ultralytics and put it in the eece490proj Folder
# Also Place the .yaml file(code in cell below) also in the xView Folder

# load a pretrained model onto GPU
model = YOLO('yolov8l.pt') 
freeze = 22 #backbone layers
freeze = [f'model.{x}.' for x in range(freeze)] # layers to freeze
count = 0 
all = 0
for k, v in model.model.named_parameters():
    all+=1
    v.requires_grad = True # train all layers
    if any(x in k for x in freeze):
        v.requires_grad = False
    else :
        count+=1
print(f'total layers: {all}, trainable layers: {count}, percentage: {count*100/all}%')
## Train the model(REMOVE BEFORE SUBMITTING->before training iza guys can check how to store and use the results to be used later inno ka variable results shu mafrood na3mol ba3den)
results = model.train(data='C://eece490proj//xView//xView.yaml', epochs=15, imgsz=960, device=0, batch =4, save = True, project = 'C://eece490proj//xView//YoloTrainingOutput' )


#IF model.train results in a FileNotFoundError there is an issue with ur paths
#IF model.train results in a NotImplementedError there is an issue with ur Cuda/PyTorch Compatibility-> run !pip3 install torch==2.2.0+cu121 torchvision==0.17.0+cu121 torchaudio===2.2.0+cu121 --index-url https://download.pytorch.org/whl/cu121 

total layers: 304, trainable layers: 49, percentage: 16.11842105263158%
Ultralytics YOLOv8.2.2  Python-3.12.2 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8l.pt, data=C://eece490proj//xView//xView.yaml, epochs=15, time=None, patience=100, batch=4, imgsz=960, save=True, save_period=-1, cache=False, device=0, workers=8, project=C://eece490proj//xView//YoloTrainingOutput, name=train17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

train: Scanning C:\eece490proj\xView\train\labels.cache... 372 images, 0 backgrounds, 0 corrupt: 100%|██████████| 372/372 [00:00<?, ?it/s]

train: WARNING  C:\eece490proj\xView\train\images\10.tif: 63 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\100.tif: 135 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\102.tif: 1965 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\1036.tif: 265 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\1037.tif: 615 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\104.tif: 1351 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\1042.tif: 483 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\1044.tif: 215 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\1046.tif: 69 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\1048.tif: 66 duplicate labels removed
train: WARNING  C:\eece490proj\xView\train\images\1049.tif: 88 duplicate labels removed
train: WARNING  C:\eece490pr


val: Scanning C:\eece490proj\xView\val\labels.cache... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<?, ?it/s]

val: WARNING  C:\eece490proj\xView\val\images\73.tif: 1056 duplicate labels removed


...

      15/15      30.1G       2.08       1.51      1.105       1348        960: 100%|██████████| 93/93 [19:08<00:00, 12.35s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  9.22it/s]

                   all          1       1056     0.0983     0.0508     0.0674     0.0396



15 epochs completed in 3.958 hours.
Optimizer stripped from C:\eece490proj\xView\YoloTrainingOutput\train17\weights\last.pt, 87.8MB
Optimizer stripped from C:\eece490proj\xView\YoloTrainingOutput\train17\weights\best.pt, 87.8MB

Validating C:\eece490proj\xView\YoloTrainingOutput\train17\weights\best.pt...
Ultralytics YOLOv8.2.2  Python-3.12.2 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
Model summary (fused): 268 layers, 43652868 parameters, 0 gradients, 165.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.42it/s]


                   all          1       1056     0.0977     0.0516     0.0729     0.0436
             Small Car          1        670      0.203    0.00448     0.0921     0.0158
                   Bus          1         14          0          0          0          0
         Utility Truck          1         11          0          0          0          0
                 Truck          1        112          0          0          0          0
           Cargo Truck          1         29          0          0          0          0
           Truck w/Box          1         11          0          0          0          0
               Trailer          1          2          0          0          0          0
       Truck w/Flatbed          1          1          0          0          0          0
        Truck w/Liquid          1          1          0          0          0          0
             Motorboat          1          4          0          0          0          0
            Dump Truc

**HERE WE ARE PRINTING THE BEST MODEL WE GOT TO BE ABLE TO VISUALIZE THE LAYERS**

In [5]:
from ultralytics import YOLO
model = YOLO('xView\\YoloTrainingOutput\\train17\\weights\\best.pt')
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_r

**HERE ARE OUR CLASSES**

In [5]:
'''

# Classes
names:
  0: Fixed-wing Aircraft
  1: Small Aircraft
  2: Cargo Plane
  3: Helicopter
  4: Passenger Vehicle
  5: Small Car
  6: Bus
  7: Pickup Truck
  8: Utility Truck
  9: Truck
  10: Cargo Truck
  11: Truck w/Box
  12: Truck Tractor
  13: Trailer
  14: Truck w/Flatbed
  15: Truck w/Liquid
  16: Crane Truck
  17: Railway Vehicle
  18: Passenger Car
  19: Cargo Car
  20: Flat Car
  21: Tank car
  22: Locomotive
  23: Maritime Vessel
  24: Motorboat
  25: Sailboat
  26: Tugboat
  27: Barge
  28: Fishing Vessel
  29: Ferry
  30: Yacht
  31: Container Ship
  32: Oil Tanker
  33: Engineering Vehicle
  34: Tower crane
  35: Container Crane
  36: Reach Stacker
  37: Straddle Carrier
  38: Mobile Crane
  39: Dump Truck
  40: Haul Truck
  41: Scraper/Tractor
  42: Front loader/Bulldozer
  43: Excavator
  44: Cement Mixer
  45: Ground Grader
  46: Hut/Tent
  47: Shed
  48: Building
  49: Aircraft Hangar
  50: Damaged Building
  51: Facility
  52: Construction Site
  53: Vehicle Lot
  54: Helipad
  55: Storage Tank
  56: Shipping container lot
  57: Shipping Container
  58: Pylon
  59: Tower

'''